In [1]:
import os
import pandas as pd
from glob import glob
root = r'F:\20230904-LiuHui\patches'
task = 'Task1'

patches = pd.DataFrame(glob(os.path.join(root, '*', '*.jpg')), columns=['fpath'])
patches['ID'] = patches['fpath'].map(lambda x: os.path.basename(os.path.dirname(x))[:15])
patches['filename'] = patches['fpath'].map(lambda x: os.path.basename(x))

In [2]:
def mapID(x):
    items = x[:-2].split('-')
    return f"20{items[0]}-{int(items[1]):05d}"

info = pd.read_csv(os.path.join(root, 'patch_info.csv'))
info['label'] = info['label'].astype(str)
info['filename'] = info['filename'].map(lambda x: os.path.basename(x))
patches = pd.merge(patches, info, on='filename', how='inner')
patches['ID'] = patches['ID'].map(mapID)
patches

,fpath,ID,filename,label_id,label,info
0,F:\20230904-LiuHui\patches\13-10097A1\13-10097...,2013-10097,13-10097A1_52224_42496_10240_27136_10752_27648...,0,_background_,"{""_background_"": 1.0}"
1,F:\20230904-LiuHui\patches\13-10097A1\13-10097...,2013-10097,13-10097A1_52224_42496_10240_27648_10752_28160...,0,_background_,"{""_background_"": 1.0}"
2,F:\20230904-LiuHui\patches\13-10097A1\13-10097...,2013-10097,13-10097A1_52224_42496_10240_28160_10752_28672...,0,_background_,"{""_background_"": 1.0}"
3,F:\20230904-LiuHui\patches\13-10097A1\13-10097...,2013-10097,13-10097A1_52224_42496_10240_28672_10752_29184...,0,_background_,"{""_background_"": 1.0}"
4,F:\20230904-LiuHui\patches\13-10097A1\13-10097...,2013-10097,13-10097A1_52224_42496_10240_29184_10752_29696...,0,_background_,"{""_background_"": 1.0}"
...,...,...,...,...,...,...
799817,F:\20230904-LiuHui\patches\14-9963A1\14-9963A1...,2014-09963,14-9963A1_44032_38400_9728_7680_10240_8192.jpg,0,_background_,"{""_background_"": 1.0}"
799818,F:\20230904-LiuHui\patches\14-9963A1\14-9963A1...,2014-09963,14-9963A1_44032_38400_9728_8192_10240_8704.jpg,0,_background_,"{""_background_"": 0.6533203125, ""_default_"": 0...."
799819,F:\20230904-LiuHui\patches\14-9963A1\14-9963A1...,2014-09963,14-9963A1_44032_38400_9728_8704_10240_9216.jpg,1,_default_,"{""_background_"": 0.033447265625, ""_default_"": ..."
799820,F:\20230904-LiuHui\patches\14-9963A1\14-9963A1...,2014-09963,14-9963A1_44032_38400_9728_9216_10240_9728.jpg,1,_default_,"{""_default_"": 1.0}"


In [7]:
import pandas as pd
import os
from onekey_algo import get_param_in_cwd
import numpy as np
label_data = pd.read_csv('data/ER.csv')
label_data

,ID,label
0,2013-00480,1
1,2013-01345,1
2,2013-03710,1
3,2013-05470,1
4,2013-05701,1
...,...,...
199,2014-45920,0
200,2014-46130,0
201,2014-46173,0
202,2014-46927,0


# 弱监督

In [23]:
import pandas as pd
import os
from onekey_algo.custom.components.comp2 import split_dataset4sol

root = r"E:\20230904-LiuHui"
save_dir = rf'E:\20230904-LiuHui\{task}'

ds = split_dataset4sol(label_data, y_data=label_data['label'], n_trails=5, save_dir=os.path.join(root, task), 
                       random_state=0, cv=False, test_size=0.3)

for idx, (train, test) in enumerate(ds):
    train = pd.merge(train, patches[['ID', 'fpath']], on='ID', how='inner')
    train[['fpath', 'label']].to_csv(os.path.join(root, task, 'split_info', f'train-RND-{idx}.txt'), index=False, header=False, sep='\t')
    test = pd.merge(test, patches[['ID', 'fpath']], on='ID', how='inner')
    test[['fpath', 'label']].to_csv(os.path.join(root, task, 'split_info', f'val-RND-{idx}.txt'), index=False, header=False, sep='\t')

# 有监督

In [8]:
import pandas as pd
import os
from onekey_algo.custom.components.comp2 import split_dataset4sol

root = r"E:\20230904-LiuHui"
save_dir = rf'E:\20230904-LiuHui\{task}'

patches_sup = patches[patches['label_id'] != 0]
ds = split_dataset4sol(label_data, y_data=label_data['label'], n_trails=5, save_dir=os.path.join(root, task), 
                       random_state=0, cv=False, test_size=0.3)

for idx, (train, test) in enumerate(ds):
    train = pd.merge(train, patches_sup[['ID', 'fpath']], on='ID', how='inner')
    train[['fpath', 'label']].to_csv(os.path.join(root, task, 'split_info', f'train-RND-{idx}.txt'), index=False, header=False, sep='\t')
    test = pd.merge(test, patches_sup[['ID', 'fpath']], on='ID', how='inner')
    test[['fpath', 'label']].to_csv(os.path.join(root, task, 'split_info', f'val-RND-{idx}.txt'), index=False, header=False, sep='\t')